<div>
    <div style="float:left;">
        <img src="http://oproject.org/tiki-download_file.php?fileId=8&display&x=450&y=128" width="50%" />
    </div>
    <div style="float:left;">
        <img src="http://gfif.udea.edu.co/root/tmva/img/tmva_logo.gif" width="50%"/>
    </div>
</div>

# JsMVA
<hr style="border-top-width: 4px; border-top-color: #34609b;">

<script src="JsRoot/scripts/JSRootCore.js?jq2d&onload=JsRootLoadedCall" type="text/javascript"></script>

In [1]:
import ROOT
from ROOT import TFile, TMVA, TCut

Welcome to JupyROOT 6.07/07


In [2]:
import sys, os
sys.path.append(os.path.expanduser("~/code/GSOC/wd/src/python"))
import JsMVA

In [3]:
%jsmva on

In [4]:
infname     = "files/tmva_class_example.root"
dataset     = "files/tmva_class_example"
treeNameSig = "TreeS"
treeNameBkg = "TreeB"
outfname    = "files/TMVA.root"
methods     = "Cuts,CutsD,CutsPCA,CutsGA,CutsSA,Likelihood,LikelihoodD,LikelihoodPCA,LikelihoodKDE,LikelihoodMIX,\
PDERS,PDERSD,PDERSPCA,PDEFoam,PDEFoamBoost,KNN,LD,Fisher,FisherG,BoostedFisher,HMatrix,FDA_GA,FDA_SA,FDA_MC,FDA_MT,\
FDA_GAMT,FDA_MCMT,MLP,MLPBFGS,MLPBNN,CFMlpANN,TMlpANN,SVM,BDT,BDTD,BDTG,BDTB,RuleFit"
verbose     = True

In [5]:
methods = "Cuts, CutsD, Likelihood, LikelihoodPCA, PDERS, PDEFoam, KNN, LD, MLPBNN, SVM, BDT, RuleFit"
methods = "BDT, MLP, KNN, LD"
methods = "BDT, MLP"
mlist = methods.replace(' ',',').split(',')
print "=== TMVAClassification: use method(s)..."
for m in mlist:
    if m.strip() != '':
        print "=== - <%s>" % m.strip()

=== TMVAClassification: use method(s)...
=== - <BDT>
=== - <MLP>


In [6]:
outputFile = TFile( outfname, 'RECREATE' )

TMVA.Tools.Instance()

factory = TMVA.Factory( "TMVAClassification", outputFile, 
                            "!V:!Silent:Color:DrawProgressBar:Transformations=I;D;P;G,D:AnalysisType=Classification" )

# Set verbosity
factory.SetVerbose( verbose )

loader = TMVA.DataLoader(dataset)

loader.AddVariable( "myvar1 := var1+var2", 'F' )
loader.AddVariable( "myvar2 := var1-var2", "Expression 2", 'F' )
loader.AddVariable( "var3",                "Variable 3", 'F' )
loader.AddVariable( "var4",                "Variable 4", 'F' )

loader.AddSpectator( "spec1:=var1*2",  "Spectator 1",  'F' )
loader.AddSpectator( "spec2:=var1*3",  "Spectator 2",  'F' )

--- Factory                  : You are running ROOT Version: 6.07/07, Apr 1, 2016
--- Factory                  : 
--- Factory                  : _/_/_/_/_/ _|      _|  _|      _|    _|_|   
--- Factory                  :    _/      _|_|  _|_|  _|      _|  _|    _| 
--- Factory                  :   _/       _|  _|  _|  _|      _|  _|_|_|_| 
--- Factory                  :  _/        _|      _|    _|  _|    _|    _| 
--- Factory                  : _/         _|      _|      _|      _|    _| 
--- Factory                  : 
--- Factory                  : ___________TMVA Version 4.2.1, Feb 5, 2015
--- Factory                  : 


In [7]:
if ROOT.gSystem.AccessPathName( "./"+infname ) != 0: 
    ROOT.gSystem.Exec( "cd files; wget https://root.cern.ch/" + infname)

In [8]:
input = TFile.Open( infname )

# Get the signal and background trees for training
signal      = input.Get( treeNameSig )
background  = input.Get( treeNameBkg )
    
# Global event weights (see below for setting event-wise weights)
signalWeight     = 1.0
backgroundWeight = 1.0

signalWeight     = 1.0
backgroundWeight = 1.0

mycuts = TCut("")
mycutb = TCut("")

loader.AddSignalTree(signal, signalWeight)
loader.AddBackgroundTree(background, backgroundWeight)
loader.fSignalWeight = signalWeight
loader.fBackgroundWeight = backgroundWeight
loader.fTreeS = signal
loader.fTreeB = background
loader.PrepareTrainingAndTestTree(mycuts,
                                  mycutb,
                                "nTrain_Signal=0:nTrain_Background=0:SplitMode=Random:NormMode=NumEvents:!V");
#"nTrain_Signal=3000:nTrain_Background=3000:nTest_Signal=1449:
#nTest_Background=1449:SplitMode=Random:NormMode=NumEvents:!V")

--- DataSetInfo              : Dataset[files/tmva_class_example] : Added class "Signal"	 with internal class number 0
--- files/tmva_class_example : Add Tree TreeS of type Signal with 6000 events
--- DataSetInfo              : Dataset[files/tmva_class_example] : Added class "Background"	 with internal class number 1
--- files/tmva_class_example : Add Tree TreeB of type Background with 6000 events
--- files/tmva_class_example : Preparing trees for training and testing...


In [9]:
# Booking selected methods, I have a function for that because booking all methods is long...
from BM import BM
BM(TMVA,factory,loader, mlist);

--- Factory                  : Booking method: MLP DataSet Name: files/tmva_class_example
--- MLP                      : Dataset[files/tmva_class_example] : Create Transformation "N" with events from all classes.
--- Norm                     : Transformation, Variable selection : 
--- Norm                     : Input : variable 'myvar1' (index=0).   <---> Output : variable 'myvar1' (index=0).
--- Norm                     : Input : variable 'myvar2' (index=1).   <---> Output : variable 'myvar2' (index=1).
--- Norm                     : Input : variable 'var3' (index=2).   <---> Output : variable 'var3' (index=2).
--- Norm                     : Input : variable 'var4' (index=3).   <---> Output : variable 'var4' (index=3).
--- MLP                      : Building Network
--- MLP                      : Initializing weights
--- Factory                  : Booking method: BDT DataSet Name: files/tmva_class_example
--- DataSetFactory           : Dataset[files/tmva_class_example] : Splitmode is:

In [10]:
factory.TrainAllMethods2(dataset)

In [ ]:
from threading import Thread

TMVA.Factory.TrainAllMethods._threaded = True
t=Thread(target=TMVA.Factory.TrainAllMethods, args=[factory])
t.start()
print("hi")

In [ ]:
#factory.TestAllMethods()

#factory.EvaluateAllMethods()

# Input variables

## Original input

In [ ]:
loader.DrawInputVariable("myvar1")

## Transformed input

In [ ]:
loader.DrawInputVariable("myvar1", processTrfs="D") #I;N;D;P;U;G,D

## Correlation matrix

In [ ]:
loader.DrawCorrelationMatrix("Signal")

# Classifier Output Distributions

In [ ]:
factory.DrawOutputDistribution(dataset, "MLP")

# Classifier Probability Distributions

In [ ]:
factory.DrawProbabilityDistribution(dataset, "LD")

# ROC curve

In [ ]:
print(dataset)
factory.DrawROCCurve(dataset)#"tmva_class_example")

# Classifier Cut Efficiencies

In [ ]:
factory.DrawCutEfficiencies(dataset, "KNN")

# Neural Network

* Mouseover (node, weight): focusing
* Zooming and grab and move supported
* Reset: double click

In [ ]:
factory.DrawNeuralNetwork(dataset, "MLP")

# Decision Tree

* Mouseover (node, weight): showing decision path
* Zooming and grab and move supported
* Reset: double click
* Click on node: 
    * hiding subtree, if node children are hidden the node will have a green border
    * rescaling: bigger nodes, bigger texts
    * click again to show the subtree

In [ ]:
factory.DrawDecisionTree(dataset, "BDT") #11

In [ ]:
outputFile.Close()